# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [70]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from datetime import datetime

# Import API key
from api_keys import geoapify_key

In [71]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data

city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,puerto natales,-51.7236,-72.4875,13.25,33,75,3.09,CL,1701109155
1,1,ilulissat,69.2167,-51.1000,-1.99,69,75,6.69,GL,1701109156
2,2,tamanrasset,22.7850,5.5228,18.95,11,0,2.06,DZ,1701109157
3,3,westport,41.1415,-73.3579,11.90,58,20,7.72,US,1701108953
4,4,waitangi,-43.9535,-176.5597,11.01,94,99,0.45,NZ,1701109157



---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [72]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color = "City",
    scale = 0.5,
    frame_width = 800,
    frame_height = 600,
    alpha = 0.75)

# Display the map
map_plot_1


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [81]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 29) &
                        (city_data_df["Max Temp"] > 20
                        )]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5.5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,27,port mathurin,-19.6833,63.4167,24.89,71,0,5.19,MU,1701109166
115,115,seminole,28.7086,-81.2081,21.31,68,0,3.60,US,1701109196
138,138,isla vista,34.4133,-119.8610,22.39,26,0,2.06,US,1701109202
179,179,beira,-19.8436,34.8389,25.05,83,0,4.63,MZ,1701109220
240,240,pisco,-13.7000,-76.2167,23.03,64,0,4.63,PE,1701109247
287,287,taoudenni,22.6783,-3.9836,22.40,13,0,3.80,ML,1701109259



### Step 3: Create a new DataFrame called `hotel_df`.

In [82]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
humidity = city_data_df["Humidity"]

hotel_df = pd.DataFrame({"City": city,
                        "Country": country,
                        "Lat": lat,
                        "Lng": lng,
                        "Humidity": humidity})
hotel_df
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
27,port mathurin,MU,-19.6833,63.4167,71,
115,seminole,US,28.7086,-81.2081,68,
138,isla vista,US,34.4133,-119.8610,26,
179,beira,MZ,-19.8436,34.8389,83,
240,pisco,PE,-13.7000,-76.2167,64,
287,taoudenni,ML,22.6783,-3.9836,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [83]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get lat, lon from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lat},{lng},{radius}"
    params["bias"] = f"proximity:{lat},{lng}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
port mathurin - nearest hotel: No hotel found
seminole - nearest hotel: No hotel found
isla vista - nearest hotel: No hotel found
beira - nearest hotel: No hotel found
pisco - nearest hotel: No hotel found
taoudenni - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
27,port mathurin,MU,-19.6833,63.4167,71,No hotel found
115,seminole,US,28.7086,-81.2081,68,No hotel found
138,isla vista,US,34.4133,-119.8610,26,No hotel found
179,beira,MZ,-19.8436,34.8389,83,No hotel found
240,pisco,PE,-13.7000,-76.2167,64,No hotel found
287,taoudenni,ML,22.6783,-3.9836,13,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [84]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    size = "Humidity",
    color="City",
    scale=0.5,
    frame_width = 800,
    frame_height = 600,
    alpha=.75,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)